In [1]:
import os, sys

# adjust to point at your repo’s scripts folder
# REPO_ROOT = os.path.abspath("/home/hice1/snachimuthu7/CSE7850/class_project/LLM-PLM-Hybrid-Project")
# SCRIPTS   = os.path.join(REPO_ROOT, "scripts")

# if SCRIPTS not in sys.path:
#     sys.path.insert(0, SCRIPTS)

# print("scripts folder:", SCRIPTS)


In [2]:
# 2.1 Create classification.csv and regression.csv
from create_corpus import main as create_corpus
create_corpus()


🔄 Attempting STREAM endpoint…
❌ STREAM failed (403 Client Error:  for url: https://rest.uniprot.org/uniprotkb/stream?query=%2A&format=tsv&fields=accession%2Csequence%2Cprotein_existence%2Cft_mod_res&compressed=true), falling back to JSON…
📄 Page 1: 500 entries
📄 Page 2: 500 entries
📄 Page 3: 500 entries
📄 Page 4: 500 entries
📄 Page 5: 500 entries
📄 Page 6: 500 entries
📄 Page 7: 500 entries
📄 Page 8: 500 entries
📄 Page 9: 500 entries
📄 Page 10: 500 entries
📄 Page 11: 500 entries
📄 Page 12: 500 entries
📄 Page 13: 500 entries
📄 Page 14: 500 entries
📄 Page 15: 500 entries
📄 Page 16: 500 entries
📄 Page 17: 500 entries
📄 Page 18: 500 entries
📄 Page 19: 500 entries
📄 Page 20: 500 entries
📄 Page 21: 500 entries
📄 Page 22: 500 entries
📄 Page 23: 500 entries
📄 Page 24: 500 entries
📄 Page 25: 500 entries
📄 Page 26: 500 entries
📄 Page 27: 500 entries
📄 Page 28: 500 entries
📄 Page 29: 500 entries
📄 Page 30: 500 entries
📄 Page 31: 500 entries
📄 Page 32: 500 entries
📄 Page 33: 500 entries
📄 Page 34: 

In [3]:

import split_corpus  # module’s top‐level code will execute splitting


✔️ classification.csv: 35000/7500/7500 train/val/test rows
✔️ regression.csv: 35000/7500/7500 train/val/test rows


In [6]:
# from generate_embeddings import process_split

# splits = [
#     "classification_train", "classification_val", "classification_test",
#     "regression_train",     "regression_val",     "regression_test"
# ]

# for s in splits:
#     print(f"Embedding split: {s}")
#     process_split(s)
!python generate_embeddings.py


🔄 Starting embedding generation…
📦 Loading ESM-2 model…
✅ ESM-2 ready on cuda
📥 Loading classification_train.csv…
 48%|█████████████████▎                  | 16772/35000 [02:25<02:34, 118.13it/s]/home/hice1/snachimuthu7/CSE7850/class_project/LLM-PLM-Hybrid-Project/scripts/generate_embeddings.py:41: UserWarning: 🔥 OOM on GPU, retrying on CPU
  warnings.warn("🔥 OOM on GPU, retrying on CPU")
100%|█████████████████████████████████████| 35000/35000 [06:03<00:00, 96.17it/s]
✅ Saved embeddings for classification_train
📥 Loading classification_val.csv…
100%|██████████████████████████████████████| 7500/7500 [01:04<00:00, 116.03it/s]
✅ Saved embeddings for classification_val
📥 Loading classification_test.csv…
  5%|█▊                                     | 357/7500 [00:03<01:02, 114.19it/s]/home/hice1/snachimuthu7/CSE7850/class_project/LLM-PLM-Hybrid-Project/scripts/generate_embeddings.py:41: UserWarning: 🔥 OOM on GPU, retrying on CPU
  warnings.warn("🔥 OOM on GPU, retrying on CPU")
100%|██████████

In [9]:
# This will write pe.pt and ptm.pt under tiny_heads/
!python train_heads.py


🔄 Starting tiny-heads training…
📥 Loading embeddings/classification_train.npz…
📥 Loading embeddings/classification_val.npz…
ℹ️ emb dim: 320
🏷️ Epoch 01: train=1.5911, val=1.5130
✅ Saved best pe head (loss=1.5130)
🏷️ Epoch 02: train=1.5132, val=1.4379
✅ Saved best pe head (loss=1.4379)
🏷️ Epoch 03: train=1.4374, val=1.3652
✅ Saved best pe head (loss=1.3652)
🏷️ Epoch 04: train=1.3641, val=1.2946
✅ Saved best pe head (loss=1.2946)
🏷️ Epoch 05: train=1.2930, val=1.2256
✅ Saved best pe head (loss=1.2256)
🏷️ Epoch 06: train=1.2235, val=1.1577
✅ Saved best pe head (loss=1.1577)
🏷️ Epoch 07: train=1.1550, val=1.0905
✅ Saved best pe head (loss=1.0905)
🏷️ Epoch 08: train=1.0873, val=1.0241
✅ Saved best pe head (loss=1.0241)
🏷️ Epoch 09: train=1.0204, val=0.9588
✅ Saved best pe head (loss=0.9588)
🏷️ Epoch 10: train=0.9546, val=0.8952
✅ Saved best pe head (loss=0.8952)
🎉 Head training complete!
📥 Loading embeddings/regression_train.npz…
📥 Loading embeddings/regression_val.npz…
🏷️ Epoch 01: train=0

In [12]:
# from faiss_index import build_faiss_index
# build_faiss_index(
#     emb_file="embeddings/classification_train.npz",
#     idx_file="embeddings/classification_train.index"
# )

!python faiss_index.py


🔄 Loading embeddings/classification_train.npz…
ℹ️ Squeezing singleton dimension: (35000, 1, 320) → (35000, 320)
ℹ️ Loaded (35000, 320) (n_vectors, dim)
⚙️ Normalizing & building index…
✅ Index saved to embeddings/classification_train.index


In [16]:
# from rag_pipeline import answer

# examples = [
#     "What is the protein existence level of lactase?",
#     "How many predicted PTM sites are there for P09812?",
#     "PTM count for MVHFAELVK?",
#     "Level of protein existence for ACDEFGHIKL?",
#     "What is the existence level of mouse hemoglobin?",
#     "PTM count for E. coli DnaK?"
# ]

# for q in examples:
#     print("▶", q)
#     print("→", answer(q), "\n")

!python rag_pipeline.py


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


E0000 00:00:1745593552.483861 1409461 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745593552.488851 1409461 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
📦 Loading intent model (BioBERT)…
✅ BioBERT intent model ready
📦 Loading ESM-2 embedder…
✅ ESM-2 ready on cuda
📦 Loading tiny_heads models…
✅ tiny_heads loaded
▶ What is the protein existence level of lactase?
ℹ️ raw : What is the protein existence level of lactase?
ℹ️ norm: what is the protein existence level of lactase
   → task=protein_existence, acc=None, raw=no, name=lactase, org=None
🔍 Strict search 'lactase'…
✅ Found None for 'lactase'
→ ⚠️ No reviewed UniProt accession found for 'lactase'. 

▶ How many predicted PTM sites are there for P09812?
ℹ️ raw : How many predicted PTM sites are there for P09812?
ℹ️ norm: how many predicted p

In [15]:
# In one cell you can launch the API—but note this will block the kernel:
!uvicorn serve_api:app --reload --port 8000


/bin/bash: line 1: uvicorn: command not found


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
